In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/glove50d/glove.6B.50d.txt
/kaggle/input/fake-news-dataset/news.csv


In [2]:
import numpy as np
import pandas as pd
import json
import csv
import random

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers

import pprint
import tensorflow.compat.v1 as tf
from tensorflow.python.framework import ops
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
tf.disable_eager_execution()

# Reading the data
data = pd.read_csv('/kaggle/input/fake-news-dataset/news.csv')
data.head()


,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [3]:
data=data.drop(["Unnamed: 0"],axis=1)
data.head(5)

,title,text,label
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


**Encoding labels**

In [4]:
le = preprocessing.LabelEncoder()
le.fit(data['label'])
data['label']=le.transform(data['label'])

In [5]:
embedding_dim = 50
max_length = 54
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size=3000
test_portion=.1

In [6]:
title = []
text = []
labels = []
for x in range(training_size):
	title.append(data['title'][x])
	text.append(data['text'][x])
	labels.append(data['label'][x])

In [7]:
tokenizer1 = Tokenizer()
tokenizer1.fit_on_texts(title)
word_index1 = tokenizer1.word_index
vocab_size1 = len(word_index1)
sequences1 = tokenizer1.texts_to_sequences(title)
padded1 = pad_sequences(
	sequences1, padding=padding_type, truncating=trunc_type)
split = int(test_portion * training_size)
test_sequences1 = padded1[0:split]
training_sequences1 = padded1[split:training_size]
test_labels = labels[0:split]
training_labels = labels[split:training_size]

In [8]:
embeddings_index = {};
with open('/kaggle/input/glove50d/glove.6B.50d.txt') as f:
    for line in f:
        values = line.split();
        word = values[0];
        coefs = np.asarray(values[1:], dtype='float32');
        embeddings_index[word] = coefs;

# Generating embeddings
embeddings_matrix = np.zeros((vocab_size1+1, embedding_dim));
for word, i in word_index1.items():
    embedding_vector = embeddings_index.get(word);
    if embedding_vector is not None:
        embeddings_matrix[i] = embedding_vector;

In [9]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size1+1, embedding_dim, input_length=max_length, weights=[embeddings_matrix], trainable=False),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv1D(64, 5, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=4),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 54, 50)            377600    
_________________________________________________________________
dropout (Dropout)            (None, 54, 50)            0         
_________________________________________________________________
conv1d (Conv1D)              (None, 50, 64)            16064     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 12, 64)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 64)                33024     
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 426,753
Trainable params: 49,153
Non-trainable params: 377,600
_____________________________________________

In [ ]:
num_epochs = 50

training_padded = np.array(training_sequences1)
training_labels = np.array(training_labels)
testing_padded = np.array(test_sequences1)
testing_labels = np.array(test_labels)

history = model.fit(training_padded, training_labels, epochs=num_epochs, validation_data=(testing_padded, testing_labels), verbose=2)

print("Training Complete")

Train on 2700 samples, validate on 300 samples
Epoch 1/50
2700/2700 - 2s - loss: 0.6404 - accuracy: 0.6252 - val_loss: 0.5921 - val_accuracy: 0.6800
Epoch 2/50
2700/2700 - 2s - loss: 0.5675 - accuracy: 0.7026 - val_loss: 0.5500 - val_accuracy: 0.7100
Epoch 3/50
2700/2700 - 2s - loss: 0.5339 - accuracy: 0.7333 - val_loss: 0.5053 - val_accuracy: 0.7333
Epoch 4/50
2700/2700 - 2s - loss: 0.4773 - accuracy: 0.7637 - val_loss: 0.5120 - val_accuracy: 0.7500
Epoch 5/50
2700/2700 - 2s - loss: 0.4407 - accuracy: 0.7996 - val_loss: 0.4736 - val_accuracy: 0.7500
Epoch 6/50
2700/2700 - 2s - loss: 0.3804 - accuracy: 0.8267 - val_loss: 0.5031 - val_accuracy: 0.7567
Epoch 7/50
2700/2700 - 2s - loss: 0.3392 - accuracy: 0.8489 - val_loss: 0.4866 - val_accuracy: 0.7733
Epoch 8/50
2700/2700 - 2s - loss: 0.3107 - accuracy: 0.8615 - val_loss: 0.4818 - val_accuracy: 0.7633
Epoch 9/50
2700/2700 - 2s - loss: 0.2890 - accuracy: 0.8796 - val_loss: 0.5056 - val_accuracy: 0.7467
Epoch 10/50
2700/2700 - 2s - loss: 

In [ ]:
X="Karry to go to France in gesture of sympathy "

sequences = tokenizer1.texts_to_sequences([X])[0]
sequences = pad_sequences([sequences],maxlen=54 , padding=padding_type, truncating=trunc_type )
if(model.predict(sequences,verbose=0)[0][0] >= 0.5 ):
    print("This news is True")
else:
    print("This news is false")
